# fit kernel density estimator models
To model the distribution of spikes over phase 

In [6]:
%reset
%load_ext autoreload
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import collections 
import kde_spikephase as kd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from kde_spikephase import kde_spikephase_estimator_forarray
from mdl_eval_tools import bayes,kl,logloss
from pprint import pprint
from rayleigh_pr import rayleigh_pr
from scipy.signal import hilbert
from scipy.stats import sem
from sklearn.model_selection import GridSearchCV 
from sklearn.neighbors import KernelDensity
from tqdm.notebook import tqdm

sns.set(font_scale=2,style='whitegrid')

pd.options.display.max_colwidth = 100

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def get_neuron_cols(df,neuron_expression):
    """df: dataframe containing columns corresponding to neuron spiketimes"""
    
    pattern = re.compile(neuron_expression)
    neuron_cols = []
    for s in df.columns:

        matched = pattern.match(s)
        is_match = bool(matched)

        if is_match:

            neuron_cols.append(matched.string)
    
    return neuron_cols

def get_allowable_lfp_wires(neuron_id,split_on):
    """neuron_id: string, e.g. TETSPK53b
    split_on: string, e.g. 'K', you will use this to exclude the unit letter from neuron_id"""

    #figure out who is the corresponding LFP for this neuron, must be on same tetrode!
    unit_wirenum = neuron.split(split_on)[1]
    unit_wirenum = unit_wirenum[:-1]
    if len(unit_wirenum) > 2:
        unit_wirenum = unit_wirenum[:2]
    allow_wire = [int(unit_wirenum), int(unit_wirenum) + 1, int(unit_wirenum) + 2, int(unit_wirenum) + 3]
    allow_wire = [str(w) for w in allow_wire]
    allow_wire = ['0'+w if int(w)<10 else w for w in allow_wire]
    
    return allow_wire

def get_lfp_cols(df,lfp_expression):
    """df: dataframe containing columns corresponding to neuron spiketimes
    lfp_expression: string, e.g."""
#example lfp_expression =  "TETFP"+allow_wire[0]+"\Z|TETFP"+allow_wire[1]+"\Z|TETFP"+allow_wire[2]+"\Z|TETFP"+allow_wire[3]+"\Z"
        
    pattern = re.compile(lfp_expression)
    lfp_cols = []
    for s in df.columns:

        matched = pattern.match(s)
        is_match = bool(matched)

        if is_match:

            lfp_cols.append(matched.string)

    return lfp_cols

## odor sampling

##### load the dataset

In [8]:
datapath = 'python_spkphase_odorsamp/'
files = os.listdir(datapath)
files = [f for f in files if not f.startswith('.')]
files = sorted(files)

##### set up the model fitting parameters
* n_splits: the number of cross-validation folds used to find the optimal kernel bandwidth
* grid_size: the number of phase bins that fitting will take place over (you will be using the WARPing method for kernel density estimation (Hall & Wand 1996 _On the accuracy of binned kernel density estimators_)
* bw_narrow: the narrowest kernel bandwidth in the range of candidate bandwidths
* bw_wide: the widest kernel bandwidth in the range of candidate bandwidths
* n_bw: the number of candidate bandwidths you'll consider
* rhythms: the list of rhythms whose phase angles you'll fit the models to

In [9]:
#fitting params
n_splits = 7
grid_size = 1000
bw_narrow = 1/30
bw_wide = 1/.6
n_bw = 20
rhythms = ['theta','lowgamma','highgamma']

savepath = 'kde_models_ints_odorsamp/' 

In [10]:
for file in tqdm(files, position=0, desc="file", leave=True, colour='dodgerblue'): 
    
    kde_df = [] #initialize the df that will contain all of this file's data
    
    df = pd.read_csv(os.path.join(datapath,file))
    odorsamp_df = df[df['trial_segment']=='dur']

    df = [] #clear out to save memory

    quarters = list(set(odorsamp_df['quarter_labels']))

    for quarter in tqdm(quarters,position=1,desc='quarter',leave=False,colour='lavender'): 

        subq = odorsamp_df[odorsamp_df['quarter_labels']==quarter]

        if np.all(subq['accuracy'] >= .75): #need to meet the accuracy criterion
            
            neuron_cols = get_neuron_cols(subq,'TETSPK')
            
            for rhythm in tqdm(rhythms,position=2,desc='rhythm',leave=False,colour='hotpink'): 

                for neuron in tqdm(neuron_cols,position=3,desc='neuron',leave=False,colour='mediumturquoise'): 

                    cell_id = file.split('_')[0] + '_' + file.split('_')[1] + '_' + neuron

                    allow_wire = get_allowable_lfp_wires(neuron,'K')

                    #grab the lfps
                    lfp_cols = get_lfp_cols(subq,"TETFP"+allow_wire[0]+"\Z|TETFP"+allow_wire[1]+"\Z|TETFP"+allow_wire[2]+"\Z|TETFP"+allow_wire[3]+"\Z")
                    lfp_name = lfp_cols[0]

                    #create the phase columns using the hilbert transform on the filtered lfp
                    filt_name = lfp_name + 'filt_' + rhythm 

                    filtered = subq[filt_name]
                    analytic_signal = hilbert(filtered)

                    phase_name = lfp_name + 'phase_' + rhythm 
                    subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);

                    #get the unique odor position identifiers for each condition
                    condition_labels = subq.groupby(['odor_labels']).sum().index
                    
                    for condition in condition_labels:

                        #need to filter the dataset by condition now
                        odor = condition

                        d = subq[(subq['odor_labels']==odor)]
                        
                        ntrials = len(set(d['trial_labels']))
                        
                        if ntrials >= 4: 

                            #now, grab the spikes and phases to fit the kde curves
                            #grab data
                            spikes = d[neuron].values
                            phases = d[phase_name].values

                            phi_when_sp = []
                            for s,p in zip(spikes,phases):
                                if s==1: 
                                    phi_when_sp.append(p)

                            phi_when_sp = np.array(phi_when_sp)

                            #CRITERION: number of spikes must exceed the number of cross val splits for fitting kde
                            if n_splits < sum(spikes):

                                kde_phaseprob, kde, bw = kde_spikephase_estimator_forarray(phi_when_sp,n_splits,grid_size,bw_narrow,bw_wide,n_bw)

                                spike_prior = d[neuron].sum()/d.shape[0]
                                phase_prior = 1/(2*np.pi)
                                kde_spikeprob = bayes(kde_phaseprob,
                                                        spike_prior,
                                                        phase_prior
                                  )

            #                     ax.plot(np.linspace(-np.pi,np.pi,grid_size), pdf, linewidth=3, alpha=0.5, label='bw=%.2f' % bw)

                                dct = {'cell_id': cell_id,
                                       'quarter_labels': np.repeat(quarter,len(kde_spikeprob)),
                                       'rhythm': np.repeat(rhythm,len(kde_spikeprob)),
                                       'kde_spikeprob': kde_spikeprob,
                                       'kde_phaseprob': kde_phaseprob,
                                       'condition_labels': np.repeat(odor,len(kde_spikeprob))
                                      }
                                kde_df.append(pd.DataFrame(dct))
            
    kde_odor = pd.concat(kde_df)               
    if not os.path.exists(savepath): 
        os.makedirs(savepath)

    savename = file.split('_')[0] + '_' + file.split('_')[1] + '_kde_models.csv'
    kde_odor.to_csv(os.path.join(savepath,savename))
        




file:   0%|          | 0/18 [00:00<?, ?it/s]

quarter:   0%|          | 0/4 [00:00<?, ?it/s]

rhythm:   0%|          | 0/3 [00:00<?, ?it/s]

neuron:   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);


...

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);
/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);


neuron:   0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);
/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);


neuron:   0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);
/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/3408091397.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered,);


After having run this, you should have a new folder in your directory that contains one .csv file per session (18 total), each of which will contain the phase model fits for a variable number of neuron-lfp combinations. 

## approach
Now, do the same but for the time-matched approach epochs. No need to reset the fitting params so we'll skip that this time, assuming you've already run the previous cells. 

In [11]:
datapath = 'python_spkphase_approach/'
files = os.listdir(datapath)
files = [f for f in files if not f.startswith('.')]
files = sorted(files)

savepath = 'kde_models_ints_approach/' 

In [12]:
for file in tqdm(files, position=0, desc="file", leave=True, colour='dodgerblue'): 
    
    kde_df = [] #initialize the df that will contain all of this file's data
    
    df = pd.read_csv(os.path.join(datapath,file))
    odorsamp_df = df[df['trial_segment']=='dur']

    df = [] #clear out to save memory

    quarters = list(set(odorsamp_df['quarter_labels']))

    for quarter in tqdm(quarters,position=1,desc='quarter',leave=False,colour='lavender'): 

        subq = odorsamp_df[odorsamp_df['quarter_labels']==quarter]

        if np.all(subq['accuracy'] >= .75): #need to meet the accuracy criterion
            
            neuron_cols = get_neuron_cols(subq,'TETSPK')
            
            for rhythm in tqdm(rhythms,position=2,desc='rhythm',leave=False,colour='hotpink'): 

                for neuron in tqdm(neuron_cols,position=3,desc='neuron',leave=False,colour='mediumturquoise'): 

                    cell_id = file.split('_')[0] + '_' + file.split('_')[1] + '_' + neuron

                    allow_wire = get_allowable_lfp_wires(neuron,'K')

                    #grab the lfps
                    lfp_cols = get_lfp_cols(subq,"TETFP"+allow_wire[0]+"\Z|TETFP"+allow_wire[1]+"\Z|TETFP"+allow_wire[2]+"\Z|TETFP"+allow_wire[3]+"\Z")
                    lfp_name = lfp_cols[0]

                    #create the phase columns using the hilbert transform on the filtered lfp
                    filt_name = lfp_name + 'filt_' + rhythm 

                    filtered = subq[filt_name]
                    analytic_signal = hilbert(filtered)

                    phase_name = lfp_name + 'phase_' + rhythm 
                    subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);

                    #get the unique odor position identifiers for each condition
                    condition_labels = subq.groupby(['odor_labels']).sum().index
                    
                    for condition in condition_labels:

                        #need to filter the dataset by condition now
                        odor = condition

                        d = subq[(subq['odor_labels']==odor)]
                        
                        ntrials = len(set(d['trial_labels']))
                        
                        if ntrials >= 4: 

                            #now, grab the spikes and phases to fit the kde curves
                            #grab data
                            spikes = d[neuron].values
                            phases = d[phase_name].values

                            phi_when_sp = []
                            for s,p in zip(spikes,phases):
                                if s==1: 
                                    phi_when_sp.append(p)

                            phi_when_sp = np.array(phi_when_sp)

                            #CRITERION: number of spikes must exceed the number of cross val splits for fitting kde
                            if n_splits < sum(spikes):

                                kde_phaseprob, kde, bw = kde_spikephase_estimator_forarray(phi_when_sp,n_splits,grid_size,bw_narrow,bw_wide,n_bw)

                                spike_prior = d[neuron].sum()/d.shape[0]
                                phase_prior = 1/(2*np.pi)
                                kde_spikeprob = bayes(kde_phaseprob,
                                                        spike_prior,
                                                        phase_prior
                                  )

            #                     ax.plot(np.linspace(-np.pi,np.pi,grid_size), pdf, linewidth=3, alpha=0.5, label='bw=%.2f' % bw)

                                dct = {'cell_id': cell_id,
                                       'quarter_labels': np.repeat(quarter,len(kde_spikeprob)),
                                       'rhythm': np.repeat(rhythm,len(kde_spikeprob)),
                                       'kde_spikeprob': kde_spikeprob,
                                       'kde_phaseprob': kde_phaseprob,
                                       'condition_labels': np.repeat(odor,len(kde_spikeprob))
                                      }
                                kde_df.append(pd.DataFrame(dct))
            
    kde_odor = pd.concat(kde_df)               
    if not os.path.exists(savepath): 
        os.makedirs(savepath)

    savename = file.split('_')[0] + '_' + file.split('_')[1] + '_kde_models.csv'
    kde_odor.to_csv(os.path.join(savepath,savename))
        




file:   0%|          | 0/18 [00:00<?, ?it/s]

quarter:   0%|          | 0/4 [00:00<?, ?it/s]

rhythm:   0%|          | 0/3 [00:00<?, ?it/s]

neuron:   0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);


...

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);
/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);


neuron:   0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);
/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);


neuron:   0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);
/var/folders/95/7z0h1t9s5tx7km1h1t7t08kw0000gn/T/ipykernel_2045/2308228644.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subq[phase_name] = np.arctan2(analytic_signal.imag,filtered);
